In [ ]:
%store -r linklst

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
import time
import selenium
from selenium import webdriver
from urllib2 import urlopen
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
import requests
import unidecode

In [ ]:
result2 = pd.DataFrame(columns=['Index','Name','Position','Episode Name','Episode date',
                               'Duration','Episode Hearts'])

In [ ]:
def get_num_epi(soup):

    temp=soup.find('div',class_='trackListCon_title').text
    temp=unidecode.unidecode(temp).split(' ')
    return int(temp[0])

In [ ]:
def scroll(driver):
    driver.execute_script(""" 
        (function () { 
        
            var y = document.body.scrollTop; 
            var step = 100; 
            window.scroll(0, y); 
            function f() { 
                if (y < document.body.scrollHeight) { 
                    y += step; 
                    window.scroll(0, y); 
                    setTimeout(f, 120); 
                }
                else { 
                    window.scroll(0, y); 
                    document.title += "scroll-done"; 
                } 
            } 
            setTimeout(f, 1000); 
        })(); 
        """)

In [ ]:
def open_driver(link,path=r'D:\GA\chromedriver.exe'):
    global driver
    driver = webdriver.Chrome(path) #chromedriver may need to be upgraded
    driver.set_page_load_timeout(120)
    driver.get(link)
    scroll(driver)


In [ ]:
%%time
loopcount = 0
for link in linklst:
    r=requests.get(link)
    soup = BeautifulSoup(r.content,'lxml')
    name=soup.find('div',class_= 'ch_feed_info').find('h1').get('title')
    count=get_num_epi(soup)
    if count>10 :
        open_driver(link)
        if count>200:
            print 'Ranking',loopcount,name,'with',count,'episodes'
            time.sleep(240)
        else:
            time.sleep(30)
        
        html=driver.page_source
        soup=BeautifulSoup(html,'lxml')
               
    episodes=soup.find_all('div',class_= 'trackRowBig-con')
    for episode in episodes:
        title=episode.find('a').find('p').get('title')
        temp=episode.find('p',class_='trackRowBig-con-des').text
        date=temp[:10]
        dura=temp[10:18]
        heart=temp[18:]
        prepare=[loopcount,name,loopcount+1,title,date,dura,heart]
        d = pd.DataFrame([prepare],columns=['Index','Name','Position','Episode Name','Episode date',
                               'Duration','Episode Hearts'])
        result2 = result2.append(d)
    print 'Episodes for rank',loopcount+1,'show finished'
    if loopcount%10 == 0:
        time.sleep(3)
    loopcount+=1

In [ ]:
result2.to_csv('episodes.csv',encoding='utf8')

In [ ]:
result2['Name'].value_counts()